In [1]:
pip install pysindy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 4.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import random
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import pysindy as ps
from sklearn.preprocessing import StandardScaler
from dynamics_generator_2d import generate_Linear, generate_FN, generate_LV, generate_DF

#1. Data Processing

##1.1 Generating Data for Bifurcations

In [4]:
Linear_gen = generate_Linear(N = 1000, noise = None)
Linear_gen_ = generate_Linear(N = 1000, noise = 'Gaussian')
FN_gen = generate_FN(N = 1000, noise = None)
FN_gen_ = generate_FN(N = 1000, noise = 'Gaussian')
LV_gen = generate_LV(N = 1000, noise = None)
LV_gen_ = generate_LV(N = 1000, noise = 'Gaussian')
DF_gen = generate_DF(N = 1000, noise = None)
DF_gen_ = generate_DF(N = 1000, noise = 'Gaussian')
Data = np.concatenate([Linear_gen, Linear_gen_, FN_gen,FN_gen_,LV_gen,
                       LV_gen_, DF_gen, DF_gen_], axis = 0)
np.random.shuffle(Data)

/usr/local/lib/python3.10/dist-packages/scipy/integrate/_odepack_py.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [6]:
X_data, y_data = [], []
library = ps.PolynomialLibrary(degree = 4)
for series, time, label in Data:
  try:
    model_sindy = ps.SINDy(feature_library = library)
    model_sindy.fit(x = series, t = time)
    coef = model_sindy.coefficients().reshape(-1)
    X_data.append(coef)
    y_data.append(label)
  except:
    pass

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/usr/local/lib/python3.10/dist-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is t

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data,y_data, stratify = y_data, test_size = 0.1, random_state = 42)

#2. Training Process
## 2.1  Support Vector Machine

### 2.1.1 Linear SVM

In [27]:
from sklearn.svm import LinearSVC, SVC

lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train,y_train)

from sklearn.metrics import accuracy_score
y_pred = lin_clf.predict(X_train)
accuracy_score(y_train, y_pred)

0.6144444444444445

In [28]:
from sklearn.metrics import confusion_matrix    
y_pred = lin_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))                    
print(confusion_matrix(y_test, y_pred) )

0.5925
[[ 58   9  31 102]
 [ 11 139  18  32]
 [ 34  22  93  51]
 [  4   4   8 184]]


### 2.1.2 RBM SVM

In [31]:
rbf_clf = SVC(kernel = 'rbf', gamma = 0.7, C=1, max_iter = 1000)    

rbf_clf.fit(X_train,y_train)
y_pred = rbf_clf.predict(X_train)
accuracy_score(y_train, y_pred)

0.9695833333333334

In [32]:
y_pred = rbf_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))                   
print(confusion_matrix(y_test, y_pred) )

0.7175
[[117   1  81   1]
 [  0 102  97   1]
 [  0   0 179  21]
 [  0   3  21 176]]


In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## 2.2 Logistic Regression

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score


classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_train)
accuracy_score(y_train, y_pred)

0.2702777777777778

In [15]:
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.26375

## 2.3 KNN

In [16]:
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_train)
accuracy_score(y_train, y_pred)

0.8909722222222223

In [17]:
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.86625

# 2.4 Naive Bayes

In [18]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_train)
accuracy_score(y_train, y_pred)

0.28930555555555554

In [19]:
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.28

## 2.5 Decision Tree

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_train)
accuracy_score(y_train, y_pred)

0.9254166666666667

In [21]:
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.84875

## 2.6 Random Forest

In [22]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_train)
accuracy_score(y_train, y_pred)

0.9180555555555555

In [23]:
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.86125